In [5]:
# Map Castilla y León cultural agenda geolocalized
import os
import folium  # Maps library
import requests # Library to make url requests
import time
import webbrowser
import bs4 # Web Scrapping
import random
import base64

In [6]:
# Web Scrapping

#cyl = "https://datosabiertos.jcyl.es/web/es/datos-abiertos-castilla-leon.html"
cyl = "https://datosabiertos.jcyl.es/web/jcyl/RISP/es/PlantillaBuscadorContenidos/1284162104333/Dataset/1284197734502/_?p0=false&p2=Dataset&tituloc=true&q=empleos&aceptar=Buscar"
def scrap(url):
    r  = requests.get(url)

    data = r.text

    soup = bs4.BeautifulSoup(data, "html.parser")
    
    return soup
soup = scrap(cyl)

# Locating website Ofertas de Empleo dataset
empleo = [x.get('href') for x in soup.findAll('a') if 'ofertas' in x.get('href')][0]
# Locating JSON file
soup = scrap(empleo)
cyl_json = [link.get('href') for link in soup.findAll('a') if "json" in link.get('href')][0]
print(cyl_json)


https://datosabiertos.jcyl.es/web/jcyl/risp/es/empleo/ofertas-empleo/1284354353012.json


In [21]:
"""
while True:
"""   
# Requesting the file
r = requests.get(cyl_json)
    
# Having the file as a JSON object
r = r.json()

#date = r['document']['date']
empleos, geolocs = r['document']['list'], []
for i in empleos:
    if 'LocalidadPadre' in i['element']['attribute'][9]:
        geolocs.append(i)
print(geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1])

['41.401096', '-4.314916']


In [25]:
# Creating a map
centro = geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
centro_posicion = [float(centro[0]),float(centro[1])]
folium_map = folium.Map(center=centro_posicion , zoom_start = 15000)

# Creating a circle on the map
for i in geolocs:
    titulo = i['element']['attribute'][1]['text']
    fecha = i['element']['attribute'][3]['date']
    content = i['element']['attribute'][-1]['valor']
    latlong = i['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
    if latlong!= []:
        posicion = [float(latlong[0]),float(latlong[1])]
    folium.Circle(
        radius=100,
        location= posicion,
        popup= content, 
        tooltip= titulo+'\n\n'+ fecha,
        color= '#'+str(random.randint(0,999999)),
        fill=True,
        ).add_to(folium_map)

In [26]:
# Saving the map to a HTML file to be read by a web browser
pagina = folium_map.save('cyl_empleos.html')

In [27]:
# Run html in webbrowser
webbrowser.open('file://'+ os.path.realpath('cyl_empleos.html'), new = -1)

False

In [28]:
"""import IPython
file = os.path.realpath('cyl_empleos.html')
IPython.display.HTML(filename = file)"""
%load_ext folium_magic


In [29]:
%folium_map